# Chapter 13


In [ ]:
# Listing Convenience function to evaluate the effect of a relationship
def evaluate_relation(relation, value):
    # separate the population
    population = train[train[relation].eq(value)] if value < 2 else train[train[relation].ge(value)]
    p = len(population)/len(train)

    # chance to survive
    surv = population[population.Survived.eq(1)]
    p_surv = len(surv)/len(population)
    return (p, p_surv)

In [ ]:
# Additional imports from previous sections
import pandas as pd
train = pd.read_csv('train.csv')

In [ ]:
# Listing The probabilities of the different populations
print("No Sibling: {:.2f} of the passengers, survival: {:.2f}".format(*evaluate_relation("SibSp", 0)))
print("One Sibling: {:.2f} of the passengers, survival: {:.2f}".format(*evaluate_relation("SibSp", 1)))
print("More Siblings: {:.2f} of the passengers, survival: {:.2f}".format(*evaluate_relation("SibSp", 2)))
print()
print("No Parent/Child: {:.2f} of the passengers, survival: {:.2f}".format(*evaluate_relation("Parch", 0)))
print("One Parent/Child: {:.2f} of the passengers, survival: {:.2f}".format(*evaluate_relation("Parch", 1)))
print("More Parents/Children: {:.2f} of the passengers, survival: {:.2f}".format(*evaluate_relation("Parch", 2)))

## Section Turning the Problem into a Circuit

In [ ]:
# Listing Passenger no 250
train[train["PassengerId"].eq(250)]

In [ ]:
# Listing Get potential relatives
current_passenger = train[train["PassengerId"].eq(250)]
last_name = current_passenger.Name.to_string(index=False).split(',')[0]
train[train["Name"].str.contains(last_name)]

In [ ]:
# Listing A look at the data
train.info()

In [ ]:
# Listing The possible relatives of Mr. Rev. Ernest Courtenay Carter
ticket = current_passenger["Ticket"].to_string(index=False)
passengerId = current_passenger["PassengerId"]

group = train[
    train["PassengerId"].ne(passengerId) & (
    train["Name"].str.contains(last_name) |
    train["Ticket"].eq(ticket)
)]
group

In [ ]:
# Listing Encoding the passengers
# number of qubits to represent considered passengers
QUBITS = 3

def encode(pos):
    bpos = "{:0{}b}".format(pos, QUBITS)
    return bpos


number_of_rows = len(group.index)
for pos in range(0, 2**QUBITS):
    if pos >= number_of_rows:
        break

    passenger = group.iloc[[pos]]
    print(pos, encode(pos), passenger.Name.to_string(index=False)) 

In [ ]:
# Listing Equal superposition of the Hadamard gates
from qiskit import QuantumCircuit, Aer, execute
from qiskit import ClassicalRegister, QuantumRegister
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from math import asin, sqrt

RULES=2

q_pass = QuantumRegister(QUBITS, name='passengers')
q_rules = QuantumRegister(RULES, name='rules')

qc = QuantumCircuit(q_pass, q_rules)

# put passenger qubits into superposition
qc.h(q_pass)
qc.barrier()

qc.draw()

In [ ]:
# Listing The oracle‐function
def oracle(passenger, group, q_p, q_r, draw=False):
    # Create a sub-circuit
    o_qc = QuantumCircuit(q_p, q_r)

    # loop through all passengers
    for pos in range(0, 2**QUBITS):
        if pos >= len(group.index):
            break

        bpos = encode(pos)
        
        # select the state representing the passenger
        select_state(bpos, o_qc, q_p)

        # apply the rules
        apply_rules(passenger, group.iloc[[pos]], o_qc, q_p, q_r)

        # un-select the state representing the passenger
        select_state(bpos, o_qc, q_p)

        if draw:
            o_qc.barrier()

    if draw:
        return o_qc.draw()
    else:
        # We return the oracle as a gate
        Oracle = o_qc.to_gate()
        Oracle.name = "oracle"
        return Oracle

In [ ]:
# Listing The select state function
def select_state(bpos, qc, qubits):
    for i in range(0, QUBITS):
        if bpos[::-1][i] == "0":
            qc.x(qubits[i])

In [ ]:
# Listing Apply the rules
from qiskit.circuit.library import ZGate

def apply_rules(passenger, current, qc, q_p, q_r):
    # apply first rule
    if passenger.Ticket.to_string(index=False) == current.Ticket.to_string(index=False):
        qc.mcx(q_p, q_r[0])

    # apply second rule
    if passenger.Name.to_string(index=False).split(',')[0] == current.Name.to_string(index=False).split(',')[0]:
        qc.mcx(q_p, q_r[1])

    # all conditions must be met
    qc.append(ZGate().control(QUBITS+RULES-1), [*q_p, *q_r])

    # unapply second rule
    if passenger.Name.to_string(index=False).split(',')[0] == current.Name.to_string(index=False).split(',')[0]:
        qc.mcx(q_p, q_r[1])

    # unapply first rule
    if passenger.Ticket.to_string(index=False) == current.Ticket.to_string(index=False):
        qc.mcx(q_p, q_r[0])


In [ ]:
# Listing Showing the oracle circuit
oracle(current_passenger, group, q_pass, q_rules, True)

In [ ]:
# Listing The amplifier‐function
def amplifier(passenger, q_p, draw=False):
    # Create a sub-circuit
    a_qc = QuantumCircuit(q_p)
    
    a_qc.h(q_p)
    a_qc.x(q_p)
    a_qc.append(ZGate().control(QUBITS-1), q_p)
    a_qc.x(q_p)
    a_qc.h(q_p)

    if draw:
        return a_qc.draw()
    else:
        # We return the oracle as a gate
        Amplifier = a_qc.to_gate()
        Amplifier.name = "amplifier"
        return Amplifier

In [ ]:
# Listing Showing the amplifier circuit
amplifier(current_passenger, q_pass, draw=True)

In [ ]:
# Listing The search‐algorithm
qc = QuantumCircuit(q_pass, q_rules)

# put passenger qubits into superposition
qc.h(q_pass)

# Apply the oracle
qc.append(oracle(current_passenger, group, q_pass, q_rules), [*q_pass, *q_rules])

# Apply the amplifier
qc.append(amplifier(current_passenger, q_pass), q_pass)

qc.draw()

In [ ]:
# Listing Result of the search algorithm
results = execute(qc,Aer.get_backend('statevector_simulator')).result()
plot_histogram(results.get_counts())

In [ ]:
# Listing Apply the amplifier multiple times
def rounds(number_of_rounds, qc, current_passenger, group, q_pass, q_rules):
    print ("{} iterations".format(number_of_rounds))
    for i in range(0,round(number_of_rounds)):
        qc.append(oracle(current_passenger, group, q_pass, q_rules), [*q_pass, *q_rules])
        qc.append(amplifier(current_passenger, q_pass), q_pass)

In [ ]:
# Listing Search algorithm with repeated amplification
from math import pi
qc = QuantumCircuit(q_pass, q_rules)

# put passenger qubits into superposition
qc.h(q_pass)
rounds(pi*sqrt(2**QUBITS)/4, qc, current_passenger, group, q_pass, q_rules)
qc.draw()

In [ ]:
# Listing Results of the repeated amplification
results = execute(qc,Aer.get_backend('statevector_simulator')).result()
plot_histogram(results.get_counts())

In [ ]:
# Listing Running a circuit with 10 passenger‐qubits
# CAUTION: THIS CIRCUIT TAKES SOME TIME
QUBITS=10

q_pass = QuantumRegister(QUBITS, name='passengers')
q_rules = QuantumRegister(RULES, name='rules')
qc = QuantumCircuit(q_pass, q_rules)

# put passenger qubits into superposition
qc.h(q_pass)
rounds(pi*sqrt(2**QUBITS)/4, qc, current_passenger, group, q_pass, q_rules)
results = execute(qc,Aer.get_backend('statevector_simulator')).result()
"Probability of finding '100': {}".format(results.get_counts()['000000000100'])

## Section Multiple Results

In [ ]:
# Listing Prepare the search
def prepare_group(passengerId):
    current_passenger = train[train["PassengerId"].eq(passengerId)]
    last_name = current_passenger.Name.to_string(index=False).split(',')[0]
    train[train["Name"].str.contains(last_name)]

    ticket = current_passenger["Ticket"].to_string(index=False)
    passengerId = current_passenger["PassengerId"]

    group = train[
        train["PassengerId"].ne(passengerId) & (
        train["Name"].str.contains(last_name) |
        train["Ticket"].eq(ticket)
    )]
    return (current_passenger, group)

In [ ]:
# Listing Run the search for a passenger's relatives
def find_relatives(current_passenger, group):
    q_pass = QuantumRegister(QUBITS, name='passengers')
    q_rules = QuantumRegister(RULES, name='rules')
    qc = QuantumCircuit(q_pass, q_rules)

    # put passenger qubits into superposition
    qc.h(q_pass)
    rounds(pi*sqrt(2**QUBITS)/4, qc, current_passenger, group, q_pass, q_rules)
    results = execute(qc,Aer.get_backend('statevector_simulator')).result()
    return plot_histogram(results.get_counts())

In [ ]:
# Listing Search for the relatives of Mr. William Ernest Carter
QUBITS=3
(current_passenger, group) = prepare_group(391)
find_relatives(current_passenger, group)

In [ ]:
# Listing Run the search for mutliple relatives
def find_relatives(current_passenger, group, cnt_searched):
    q_pass = QuantumRegister(QUBITS, name='passengers')
    q_rules = QuantumRegister(RULES, name='rules')
    qc = QuantumCircuit(q_pass, q_rules)

    # put passenger qubits into superposition
    qc.h(q_pass)
    rounds(pi/4*sqrt(2**QUBITS/cnt_searched), qc, current_passenger, group, q_pass, q_rules)
    results = execute(qc,Aer.get_backend('statevector_simulator')).result()
    return plot_histogram(results.get_counts())

In [ ]:
# Listing Result of the search with adjusted number of iterations
find_relatives(current_passenger, group, current_passenger["SibSp"]+current_passenger["Parch"])